In [23]:
from PublicDataReader import HousingLicense
import PublicDataReader as pdr
import requests
from pprint import pprint
import json
import pandas as pd
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, quote
import re
import os

## API 키 설정해서 api 객체 생성

In [2]:
# 인코딩 API키 
# rDXCeN2Hk%2FkCO%2Bq5epV%2BZWF%2BMuPCtRUNUCBQo663B%2FYOzn7rj%2FwQ3jxs7im7BbP4GDPRahe49n3kmsM3hhQziQ%3D%3D
# 디코딩 API키
# rDXCeN2Hk/kCO+q5epV+ZWF+MuPCtRUNUCBQo663B/YOzn7rj/wQ3jxs7im7BbP4GDPRahe49n3kmsM3hhQziQ==
service_key = 'rDXCeN2Hk%2FkCO%2Bq5epV%2BZWF%2BMuPCtRUNUCBQo663B%2FYOzn7rj%2FwQ3jxs7im7BbP4GDPRahe49n3kmsM3hhQziQ%3D%3D'
api = HousingLicense(service_key)

## 시군구코드, 법정동코드 조회해서 변수에 저장하기

In [3]:
sigungu_name = "광진구"
# 광진구 읍면동 = 광장동, 구의동, 군자동, 능동, 자양동, 중곡동, 화양동
bjdong_name = "광장동"
bjdong_name_list = ["광장동", "구의동", "군자동", "능동", "자양동", "중곡동", "화양동"]

In [4]:
code_df = pdr.code_bdong()
for i in range(0,len(bjdong_name_list)):
    gcode = code_df.loc[(code_df['시군구명'].str.contains(sigungu_name)) 
#                         & (code_df['읍면동명'] == bjdong_name_list[i])
                       ]
    print("%s , %s , %s",gcode["읍면동명"],gcode["시군구코드"],gcode['법정동코드'])

출처: 행정기관(행정동) 및 관할구역(법정동) 변경내역(2023. 2. 24. 시행)
URL: https://www.mois.go.kr/frt/bbs/type001/commonSelectBoardArticle.do?bbsId=BBSMSTR_000000000052&nttId=98870

%s , %s , %s 332       
333    중곡동
334     능동
335    구의동
336    광장동
337    자양동
338    노유동
339    화양동
340    모진동
341    군자동
Name: 읍면동명, dtype: object 332    11215
333    11215
334    11215
335    11215
336    11215
337    11215
338    11215
339    11215
340    11215
341    11215
Name: 시군구코드, dtype: object 332    1121500000
333    1121510100
334    1121510200
335    1121510300
336    1121510400
337    1121510500
338    1121510600
339    1121510700
340    1121510800
341    1121510900
Name: 법정동코드, dtype: object
%s , %s , %s 332       
333    중곡동
334     능동
335    구의동
336    광장동
337    자양동
338    노유동
339    화양동
340    모진동
341    군자동
Name: 읍면동명, dtype: object 332    11215
333    11215
334    11215
335    11215
336    11215
337    11215
338    11215
339    11215
340    11215
341    11215
Name: 시군구코드, dtype: object 332    1121500000
333

In [5]:
sigungu_code = 11215
bjdong_code_list = [10400,10300,10900,10200,10500,10100,10700]

# api 사용해서 데이터 얻기

In [34]:
data_list = list()
for i in range(0,len(bjdong_code_list)):
    try:
        data_list.append(api.get_data(
            service_type="기본개요",
            sigungu_code=sigungu_code,
            bdong_code=bjdong_code_list[i]
        #     ,bun = #선택사항
        #     ,ji =  #선택사항
        ))
    except Exception:
        print("에러난 동: ",bjdong_code_list[i])

In [35]:
data_list[0].head(3)

,건물명,특수지명,블록,로트,용도코드,용도코드명,구조코드,구조코드명,주건축물수,연면적(㎡),...,사용검사예정일,생성일자,순번,대지위치,시군구코드,법정동코드,대지구분코드,번,지,관리주택대장PK
0,None,None,None,None,None,None,None,None,0,0,...,None,20130517,1,서울특별시 광진구 광장동 484번지,11215,10400,0,0484,0000,11215-100006604
1,None,None,None,None,None,None,None,None,0,0,...,None,20130726,2,서울특별시 광진구 광장동 567번지,11215,10400,0,0567,0000,11215-100006624
2,None,None,None,None,None,None,None,None,0,0,...,None,20130523,3,서울특별시 광진구 광장동 484번지,11215,10400,0,0484,0000,11215-100006644


In [38]:
data_list[1].head(2)

,건물명,특수지명,블록,로트,용도코드,용도코드명,구조코드,구조코드명,주건축물수,연면적(㎡),...,사용검사예정일,생성일자,순번,대지위치,시군구코드,법정동코드,대지구분코드,번,지,관리주택대장PK
0,구의동 청년주택,None,None,None,02000,공동주택,None,None,1,25904.66,...,20240930,20230403,1,서울특별시 광진구 구의동 246-10번지,11215,10300,0,0246,0010,11000-100007369
1,None,None,None,None,None,None,None,None,0,0,...,None,20130529,2,서울특별시 광진구 구의동 631-1번지,11215,10300,0,0631,0001,11215-100006684


### api로 받은 데이터를 파일로 저장하기

In [8]:
# data.to_csv("../data/residence_info/g10400.csv",index=False,encoding='utf-8')

In [40]:
for i in range(0,len(data_list)):
    try:
        data_list[i].to_csv("../data/residence_info/g{}.csv".format(i),index=False,encoding='utf-8')
    except Exception as e:
        print("파일 저장 중 에러 발생 :",data_list[i].head(1))
        print(e)

# GeoCoder 사용해서 번지 주소를 위경도로 변환

In [41]:
regex = r'서울특별시 광진구 \S*동 [\d|-]*'
pattern = re.compile(regex)

In [42]:
detail_addr_index = data_list[0].columns.get_loc('대지위치')
print(detail_addr_index)

23


In [43]:
vworld_api_key = "07CCF2CD-23F5-3D13-BF3D-1F7676664B6A"

In [46]:
del locInfo_list
del err_list

In [47]:
# result : 번지주소 모음 리스트
locInfo_list = list()
err_list = list()
for i in range(0,len(data_list)):
    result_addr = list()
    error_addr = list()
    for row in data_list[i].values:
        try:
            a = pattern.match(row[detail_addr_index]).group()
#             print(a)
            result_addr.append(a)
        except Exception:
            error_addr.append(a)
    print("******",len(result_addr),len(error_addr),"************************")
    for j in range(0,len(error_addr)):
        print(error_addr[j])
        if (j == len(error_addr) -1):
            print("******************************")
    locInfo = list()
    err_loc = list()
    for index,e in enumerate(result_addr):
        temp_addr = quote_plus(e)
        url = "http://api.vworld.kr/req/address?service=address&request=getcoord&version=2.0&crs=epsg:4326&address={}&refine=true&simple=false&format=json&type=parcel&key=07CCF2CD-23F5-3D13-BF3D-1F7676664B6A".format(temp_addr)
        try:
            request = Request(url)
            response = urlopen(request)
            res_code = response.getcode()
            if res_code == 200:
                response_body = response.read().decode('utf-8')
                jsonData = json.loads(response_body)
#                     pprint(jsonData)
                lat = float(jsonData['response']['result']['point']['x'])
                lng = float(jsonData['response']['result']['point']['y'])
                locInfo.append({"bunji":e,"lat":lat,"lng":lng})
            else:
                print('API error')
        except Exception:
           err_loc.append(e) 
    locInfo_list.append(locInfo)
    err_list.append(err_loc)

****** 1256 7 ************************
서울특별시 광진구 광장동 145-8
서울특별시 광진구 광장동 484
서울특별시 광진구 광장동 554-7
서울특별시 광진구 광장동 577
서울특별시 광진구 광장동 484
서울특별시 광진구 광장동 484-2
서울특별시 광진구 광장동 218-1
******************************
****** 593 0 ************************
****** 29 0 ************************
****** 1 0 ************************
****** 520 2 ************************
서울특별시 광진구 자양동 787
서울특별시 광진구 자양동 787
******************************
****** 15 1 ************************
서울특별시 광진구 중곡동 187-22
******************************
****** 11 0 ************************


In [52]:
loc_df = list()
for i in range(0,len(locInfo_list)):
    loc_df.append(pd.DataFrame(locInfo_list[i]))

In [55]:
for i in range(0,len(loc_df)):
    try:
        loc_df[i].to_csv("../data/residence_info/loc{}.csv".format(i),index=False,encoding='utf-8')
    except Exception as e:
        print("파일 저장 중 에러 발생 :",data_list[i].head(1))
        print(e)

서울특별시 광진구 광장동 145-8
서울특별시 광진구 광장동 145-8
서울특별시 광진구 광장동 145-8
서울특별시 광진구 광장동 145-8
서울특별시 광진구 광장동 145-8
서울특별시 광진구 광장동 145-8
서울특별시 광진구 광장동 145-8


### 주소 url 인코딩

In [12]:
# vworld_api_key = "07CCF2CD-23F5-3D13-BF3D-1F7676664B6A"

In [14]:
address = result_addr[0]
# url = "http://api.vworld.kr/req/address?service=address&request=getcoord&version=2.0&crs=epsg:4326&address={}&refine=true&simple=false&format=json&type=parcel&key={}".format(address,vworld_api_key)
# print(url)

In [31]:
locInfo = list()
err_loc = list()
for index,e in enumerate(result_addr):
    temp_addr = quote_plus(e)
    url = "http://api.vworld.kr/req/address?service=address&request=getcoord&version=2.0&crs=epsg:4326&address={}&refine=true&simple=false&format=json&type=parcel&key=07CCF2CD-23F5-3D13-BF3D-1F7676664B6A".format(temp_addr)
    try:
        request = Request(url)
        response = urlopen(request)
        res_code = response.getcode()
        if res_code == 200:
            response_body = response.read().decode('utf-8')
            jsonData = json.loads(response_body)
            pprint(jsonData)
            lat = float(jsonData['response']['result']['point']['x'])
            lng = float(jsonData['response']['result']['point']['y'])
            locInfo.append({"bunji":e,"lat":lat,"lng":lng})
        else:
            print('error')
    except Exception:
       err_loc.append(e) 

{'response': {'input': {'address': '서울특별시 광진구 광장동 484', 'type': 'parcel'},
              'refined': {'structure': {'detail': '광장현대3단지아파트',
                                        'level0': '대한민국',
                                        'level1': '서울특별시',
                                        'level2': '광진구',
                                        'level3': '',
                                        'level4A': '광장동',
                                        'level4AC': '1121581000',
                                        'level4L': '광장동',
                                        'level4LC': '1121510400104840000',
                                        'level5': '484'},
                          'text': '서울특별시 광진구 광장동 484'},
              'result': {'crs': 'EPSG:4326',
                         'point': {'x': '127.098376980961',
                                   'y': '37.5405244122274'}},
              'service': {'name': 'address',
                          'operation': 'getcoord',

{'response': {'input': {'address': '서울특별시 광진구 광장동 566', 'type': 'parcel'},
              'refined': {'structure': {'detail': '교육연구시설',
                                        'level0': '대한민국',
                                        'level1': '서울특별시',
                                        'level2': '광진구',
                                        'level3': '',
                                        'level4A': '광장동',
                                        'level4AC': '1121581000',
                                        'level4L': '광장동',
                                        'level4LC': '1121510400105660000',
                                        'level5': '566'},
                          'text': '서울특별시 광진구 광장동 566'},
              'result': {'crs': 'EPSG:4326',
                         'point': {'x': '127.096247918065',
                                   'y': '37.5424903339894'}},
              'service': {'name': 'address',
                          'operation': 'getcoord',
   

{'response': {'input': {'address': '서울특별시 광진구 광장동 566', 'type': 'parcel'},
              'refined': {'structure': {'detail': '교육연구시설',
                                        'level0': '대한민국',
                                        'level1': '서울특별시',
                                        'level2': '광진구',
                                        'level3': '',
                                        'level4A': '광장동',
                                        'level4AC': '1121581000',
                                        'level4L': '광장동',
                                        'level4LC': '1121510400105660000',
                                        'level5': '566'},
                          'text': '서울특별시 광진구 광장동 566'},
              'result': {'crs': 'EPSG:4326',
                         'point': {'x': '127.096247918065',
                                   'y': '37.5424903339894'}},
              'service': {'name': 'address',
                          'operation': 'getcoord',
   

{'response': {'input': {'address': '서울특별시 광진구 광장동 484', 'type': 'parcel'},
              'refined': {'structure': {'detail': '광장현대3단지아파트',
                                        'level0': '대한민국',
                                        'level1': '서울특별시',
                                        'level2': '광진구',
                                        'level3': '',
                                        'level4A': '광장동',
                                        'level4AC': '1121581000',
                                        'level4L': '광장동',
                                        'level4LC': '1121510400104840000',
                                        'level5': '484'},
                          'text': '서울특별시 광진구 광장동 484'},
              'result': {'crs': 'EPSG:4326',
                         'point': {'x': '127.098376980961',
                                   'y': '37.5405244122274'}},
              'service': {'name': 'address',
                          'operation': 'getcoord',

{'response': {'input': {'address': '서울특별시 광진구 광장동 484', 'type': 'parcel'},
              'refined': {'structure': {'detail': '광장현대3단지아파트',
                                        'level0': '대한민국',
                                        'level1': '서울특별시',
                                        'level2': '광진구',
                                        'level3': '',
                                        'level4A': '광장동',
                                        'level4AC': '1121581000',
                                        'level4L': '광장동',
                                        'level4LC': '1121510400104840000',
                                        'level5': '484'},
                          'text': '서울특별시 광진구 광장동 484'},
              'result': {'crs': 'EPSG:4326',
                         'point': {'x': '127.098376980961',
                                   'y': '37.5405244122274'}},
              'service': {'name': 'address',
                          'operation': 'getcoord',

...

{'response': {'input': {'address': '서울특별시 광진구 광장동 554-7', 'type': 'parcel'},
              'refined': {'structure': {'detail': '광장현대아파트 503동',
                                        'level0': '대한민국',
                                        'level1': '서울특별시',
                                        'level2': '광진구',
                                        'level3': '',
                                        'level4A': '광장동',
                                        'level4AC': '1121581000',
                                        'level4L': '광장동',
                                        'level4LC': '1121510400105540007',
                                        'level5': '554-7'},
                          'text': '서울특별시 광진구 광장동 554-7'},
              'result': {'crs': 'EPSG:4326',
                         'point': {'x': '127.100171995577',
                                   'y': '37.5398732591931'}},
              'service': {'name': 'address',
                          'operation': 'ge

{'response': {'input': {'address': '서울특별시 광진구 광장동 484', 'type': 'parcel'},
              'refined': {'structure': {'detail': '광장현대3단지아파트',
                                        'level0': '대한민국',
                                        'level1': '서울특별시',
                                        'level2': '광진구',
                                        'level3': '',
                                        'level4A': '광장동',
                                        'level4AC': '1121581000',
                                        'level4L': '광장동',
                                        'level4LC': '1121510400104840000',
                                        'level5': '484'},
                          'text': '서울특별시 광진구 광장동 484'},
              'result': {'crs': 'EPSG:4326',
                         'point': {'x': '127.098376980961',
                                   'y': '37.5405244122274'}},
              'service': {'name': 'address',
                          'operation': 'getcoord',

{'response': {'input': {'address': '서울특별시 광진구 광장동 484', 'type': 'parcel'},
              'refined': {'structure': {'detail': '광장현대3단지아파트',
                                        'level0': '대한민국',
                                        'level1': '서울특별시',
                                        'level2': '광진구',
                                        'level3': '',
                                        'level4A': '광장동',
                                        'level4AC': '1121581000',
                                        'level4L': '광장동',
                                        'level4LC': '1121510400104840000',
                                        'level5': '484'},
                          'text': '서울특별시 광진구 광장동 484'},
              'result': {'crs': 'EPSG:4326',
                         'point': {'x': '127.098376980961',
                                   'y': '37.5405244122274'}},
              'service': {'name': 'address',
                          'operation': 'getcoord',

{'response': {'input': {'address': '서울특별시 광진구 광장동 453-1', 'type': 'parcel'},
              'refined': {'structure': {'detail': '삼성아파트 상가동',
                                        'level0': '대한민국',
                                        'level1': '서울특별시',
                                        'level2': '광진구',
                                        'level3': '',
                                        'level4A': '광장동',
                                        'level4AC': '1121581000',
                                        'level4L': '광장동',
                                        'level4LC': '1121510400104530001',
                                        'level5': '453-1'},
                          'text': '서울특별시 광진구 광장동 453-1'},
              'result': {'crs': 'EPSG:4326',
                         'point': {'x': '127.102063542226',
                                   'y': '37.5424058873846'}},
              'service': {'name': 'address',
                          'operation': 'getco

{'response': {'input': {'address': '서울특별시 광진구 광장동 188-2', 'type': 'parcel'},
              'refined': {'structure': {'detail': '',
                                        'level0': '대한민국',
                                        'level1': '서울특별시',
                                        'level2': '광진구',
                                        'level3': '',
                                        'level4A': '',
                                        'level4AC': '',
                                        'level4L': '광장동',
                                        'level4LC': '1121510400101880002',
                                        'level5': '188-2'},
                          'text': '서울특별시 광진구 광장동 188-2'},
              'result': {'crs': 'EPSG:4326',
                         'point': {'x': '127.109492539309',
                                   'y': '37.5473967761677'}},
              'service': {'name': 'address',
                          'operation': 'getcoord',
                

In [ ]:
print(len(err_loc))
for i in range(0,len(err_loc)):
    print(err_loc[i])

In [30]:
locInfo[1]

{'lat': 127.108615954457, 'lng': 37.547168630625}